In [13]:
from pandas import *
from pathlib import Path
from jinja2 import Environment, FileSystemLoader, select_autoescape
from IPython import display
from commonmark import commonmark
from json import loads, dumps

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md']),
    trim_blocks=True,
    lstrip_blocks=True
    )

env.filters['markdown'] = markdown

In [14]:
#ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/input/"  # for pc
in_path = Path(r"/Users/ehaas/Documents/FHIR/US-Core/input/images/uscdi_table.csv")    # for mac
# out_path = Path(r"/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/includes/uscdi-uscore-map.html")    # for mac
out_path = Path(r"/Users/ehaas/Documents/FHIR/US-Core/input/includes/uscdi-uscore-map.html") # for mac
# out_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/includes/uscdi-uscore-map.html') # temp path for testing

guidance_links = {
    "See SDOH Guidance": "screening-and-assessments.html",
    "See Medication List Guidance": "medication-list.html",
    "See Assessments Guidance": "screening-and-assessments.html",
    "See Clinical Notes Guidance": "clinical-notes.html",
    "See Basic Provenance Guidance": "basic-provenance.html"
}

print(guidance_links["See Assessments Guidance"])

screening-and-assessments.html


### Get CSV Data Using Pandas

In [15]:
df = read_csv(in_path,na_filter = False)
df.head()

,data_class,data_element,uscore_profile,fhir_path,FHIR Resource,comment
0,Allergies and Intolerances,,US Core AllergyIntolerance Profile,US Core Allergies Profile,AllergyIntolerance,
1,Allergies and Intolerances,Substance (Medication),,,,
2,Allergies and Intolerances,Substance (Drug Class),,,,
3,Allergies and Intolerances,Reaction,,,,
4,Assessment and Plan of Treatment,,,US Core CarePlan Profile,CarePlan,


### update rel uris

- get profile name
  - find in /Users/ehaas/Documents/FHIR/US-Core/temp/pages/_data/structuredefinitions.json using name element
  - return path element
  - update cell as html anchor
- get comment
  - find in link in hand edited guidance_links dict
   - update cell as html anchor

In [16]:
sds_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/temp/pages/_data/structuredefinitions.json')
sds = loads(sds_path.read_text())

for i,title in df.uscore_profile.items():
    if title:
        title_split = title.split('|')
        for j,t in enumerate(title_split):
            t = t.strip()
            # print(t)
            for sd in sds:
                if sds[sd]['title'] == t:
                    # print(sds[sd]['path'])
                    title_split[j] = f'<a href="{sds[sd]["path"]}" target="_blank">{t}</a>'
        #             print(title_split[j])
        # print(title_split)
        # print('|'.join(title_split))
        df.uscore_profile[i] = '|'.join(title_split)

for i,comment in df.comment.items():

    try:
        df.comment[i] =  f'<a href="{guidance_links[comment]}" target="_blank">{comment}</a>'
    except KeyError:
        pass
df[5:10]

,data_class,data_element,uscore_profile,fhir_path,FHIR Resource,comment
5,Assessment and Plan of Treatment,Assessment and Plan of Treatment,"<a href=""StructureDefinition-us-core-careplan....",,,
6,Assessment and Plan of Treatment,SDOH Assessment,"<a href=""StructureDefinition-us-core-observati...",US Core Observation SDOH Assessment Profile|US...,"Observation, QuestionnaireResponse","<a href=""screening-and-assessments.html"" targe..."
7,Care Team Members,,"<a href=""StructureDefinition-us-core-careteam....",US Core CareTeam Profile,CareTeam.participant,
8,Care Team Members,Name,,US Core Practitioner Profile| US Core Patient ...,"Practitioner.name, Patient.name, RelatedPerson...",
9,Care Team Members,Identifier,,US Core Practitioner Profile| US Core Patient ...,"Practitioner.identifier, Patient.identifier",


### Splits DF into DFs by Data Class

In [17]:

grouped = df.groupby('data_class')  
dfs = [grouped.get_group(x).reset_index(0, drop=True) for x in grouped.groups if x != '']
data_classes = [(x) for x in grouped.groups.keys() if x != '']
print(data_classes)
print(dfs[0].shape[0])
dfs[0].head()

['Allergies and Intolerances', 'Assessment and Plan of Treatment', 'Care Team Members', 'Clinical Notes', 'Clinical Tests', 'Diagnostic Imaging', 'Encounter', 'Foo', 'Goals', 'Health Insurance Information', 'Health Status/Assessments', 'Immunizations', 'Laboratory', 'Medications', 'Patient Demographics', 'Problems', 'Procedures', 'Provenance', 'Unique Device Identifier(s)', 'Vital Signs']
4


,data_class,data_element,uscore_profile,fhir_path,FHIR Resource,comment
0,Allergies and Intolerances,,"<a href=""StructureDefinition-us-core-allergyin...",US Core Allergies Profile,AllergyIntolerance,
1,Allergies and Intolerances,Substance (Medication),,,,
2,Allergies and Intolerances,Substance (Drug Class),,,,
3,Allergies and Intolerances,Reaction,,,,


### Use DF To Populate Jinja2 Template

In [18]:
template = env.get_template('my_mapping_template.j2')   
my_html = template.render(dfs=dfs,data_classes=data_classes)
# add comment to line 1
# my_string = f'<!-- Source = {in_path}. This file is generated by SearchParameterListMaker.ipynb. Do not edit directly. -->\n\n{my_string}'
display.display_html(my_html, raw=True)
# save in pages folder
print(f'Writing {out_path}...')
out_path.write_text(my_html)    


Allergies and Intolerances,US Core AllergyIntolerance Profile,
•Substance (Medication),,
•Substance (Drug Class),,
•Reaction,,
Assessment and Plan of Treatment,,
•Assessment and Plan of Treatment,US Core CarePlan Profile,
•SDOH Assessment,US Core Observation SDOH Assessment ProfileUS Core Observation Social History ProfileUS Core QuestionnaireResponse Profile,See SDOH Guidance
Care Team Members,US Core CareTeam Profile,
•Name,,
•Identifier,,
•Location,,


Writing /Users/ehaas/Documents/FHIR/US-Core/input/includes/uscdi-uscore-map.html...


29032